In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt
import wandb
from datasets import load_metric, Dataset
from IPython.display import Markdown, display
from IPython.core.debugger import set_trace
import plotly.express as px

from datasets import load_dataset, Dataset

from lexicalrichness import LexicalRichness
import functools
import string
import psutil
from collections import defaultdict
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
import operator
import spacy
import textstat
import difflib as dl
from itertools import groupby
from operator import itemgetter

In [11]:
from analysis_functions import *

In [12]:
seed = 420
np.random.seed(seed)
run_name = "mild-jazz-178"
run_id = "36206pkc"
wandb_run_path = f"uts_nlp/travis_attack/{run_id}"  # get this from wandb overview section 
pd.options.mode.chained_assignment = None  # default='warn'


# Init W&B run object
api = wandb.Api()
run = api.run(wandb_run_path)
config = run.config

# Load CSV files
path_results = f"../model_checkpoints/travis_attack/{run_name}/"
train            = pd.read_csv(path_results + "train.csv")
valid            = pd.read_csv(path_results + "valid.csv")
test             = pd.read_csv(path_results + "test.csv")
training_step    = pd.read_csv(path_results + "training_step.csv")
training_summary = pd.read_csv(path_results + "training_summary.csv")

## Dataset-level stats

In [13]:
#df_training_step = postprocess_df(training_step, filter_idx=None)
#df_train = postprocess_df(train, filter_idx=None)
#df_valid = postprocess_df(valid, filter_idx=None)

In [14]:
#df_training_step.to_csv(path_results + "df_temp_training_step.csv",index=False)
#df_train.to_csv(path_results + "df_temp_train.csv",index=False)
#df_valid.to_csv(path_results + "df_temp_valid.csv",index=False)

df_training_step = pd.read_csv(path_results + "df_temp_training_step.csv")
df_train = pd.read_csv(path_results + "df_temp_train.csv")
df_valid = pd.read_csv(path_results + "df_temp_valid.csv")

In [15]:
df_training_step['data_split'] = 'training_step'
df_train['data_split'] = 'eval_train'
df_valid['data_split'] = 'eval_valid'
df_concat = pd.concat([df_training_step,df_train,df_valid])

# Otherwise we get a big spike at 0
df_concat.loc[df_concat.epoch_of_first_label_flip == 0, 'epoch_of_first_label_flip'] = None
fig_l = []

In [16]:
hist_config_dicts = [
    {
        'colname': 'epoch_of_first_label_flip', 
        'xlabel': "Epoch of first label flip", 
        'desc': "Cumulative prob Epoch of first label flip for each original example",
        'cumulative': True,
    },
    {
        'colname': 'idx_n_unique_pp', 
        'xlabel': "Unique paraphrases per original example", 
        "desc": "Number of generated unique paraphrases per original example during training", 
        'cumulative': False,
    },
    {
        'colname': 'idx_n_pp_changes', 
        'xlabel': "Paraphrase changes per original example", 
        "desc": "Number of paraphrase changes per original example during training", 
        'cumulative': False,
    },
]


fig_l = []
line_colnames = [o for o in df_concat.columns if "_diff" in o] + ['rouge_score', "is_truncation"]
for colname in line_colnames: 
    fig = plot_epoch_line_charts(df_concat, colname)
    fig_l.append({f"pp_metrics/{colname}":fig })
    
for d in hist_config_dicts: 
    fig = plot_idx_hist(df_concat, d['colname'],d['xlabel'],d['cumulative'])
    fig_l.append({f"pp_metrics/{d['colname']}":fig })

d1 = {k: v for d in fig_l for k, v in d.items()}
#for fig in d1.values(): fig.show()
run1 = wandb.init(job_type="analysis")
run1.log(d1)
run1.finish()

In [20]:
#display_all(df[['orig_l', 'pp_l','removals','insertions','unchanged',  'is_truncation']].sample(10))

,orig_l,pp_l,removals,insertions,unchanged,is_truncation
4613,"for a good chunk of its running time , trapped is an effective and claustrophobic thriller .","For a good chunk of its running time, trapped is an effective and claustrophobic thriller.",[for],[For],"[a good chunk of its running time, trapped is an effective and claustrophobic thriller.]",False
5013,a cop story that understands the medium amazingly well .,A cop story that understands the medium amazingly well.,[a],[A],[cop story that understands the medium amazingly well.],False
6495,"the passions aroused by the discord between old and new cultures are set against the strange , stark beauty of the mideast desert , so lovingly and perceptively filmed that you can almost taste the desiccated air .","The passions aroused by the discord between old and new cultures are set against the strange, stark beauty of the mideast desert, so lovingly and perceptively filmed that you can almost taste the desiccated air.",[the],[The],"[passions aroused by the discord between old and new cultures are set against the strange, stark beauty of the mideast desert, so lovingly and perceptively filmed that you can almost taste the desiccated air.]",False
6606,behan himself knew how to spin a tale and one can't help but think he'd appreciate this attempt to turn his life into art .,behan himself knew how to spin a tale and one can't help but think he'd appreciate this attempt to turn his life into art.,[],[],[behan himself knew how to spin a tale and one ca n't help but think he 'd appreciate this attempt to turn his life into art.],False
3278,"an enthralling , playful film that constantly frustrates our desire to know the 'truth' about this man , while deconstructing the very format of the biography in a manner that derrida would doubtless give his blessing to .","An enthralling, playful film that constantly frustrates our desire to know the 'truth' about this man, while deconstructing the very format of the biography in a manner that derrida would doubtless give his blessing to.",[an],[An],"[enthralling, playful film that constantly frustrates our desire to know the' truth' about this man, while deconstructing the very format of the biography in a manner that derrida would doubtless give his blessing to.]",False
5744,"[drumline] is entertaining for what it does , and admirable for what it doesn't do .","[drumline] is entertaining for what it does, and admirable for whatit doesn't do.","[what, it]",[whatit],"[[ drumline] is entertaining for what it does, and admirable for, does n't do.]",False
7555,"thankfully , the film , which skirts that rapidly deteriorating line between fantasy and reality . . . takes a tongue-in-cheek attitude even as it pushes the croc hunter agenda .","thankfully, the film, which skirts that rapidly deteriorating line between fantasy and reality.",[.. takes a tongue- in- cheek attitude even as it pushes the croc hunter agenda.],[],"[thankfully, the film, which skirts that rapidly deteriorating line between fantasy and reality.]",True
1800,part of the charm of satin rouge is that it avoids the obvious with humour and lightness .,Part of the charm of satin rouge is that it avoids the obvious with humour and lightness.,[part],[Part],[of the charm of satin rouge is that it avoids the obvious with humour and lightness.],False
1408,"sandra nettelbeck beautifully orchestrates the transformation of the chilly , neurotic , and self-absorbed martha as her heart begins to open .","sandra nettelbeck beautifully orchestrates the transformation of the chilly, neurotic, and self-absorbed martha as her heart begins to open.",[],[],"[sandra nettelbeck beautifully orchestrates the transformation of the chilly, neurotic, and self- absorbed martha as her heart begins to open.]",False
8058,"intriguing and beautiful film , but those of you who read the book are likely to be disappointed .","intriguing and beautiful film, but those of you who read the book are likely to be disappointed

## Individual examples 

## Text statistics (to send to w&b )